# Feature Detection

#### The SIFT & SURF algorithms are patented by their respective creators, and while they are free to use in academic and research settings, you should technically be obtaining a license/permission from the creators if you are using them in a commercial (i.e. for-profit) application.

In [2]:
import sys
import os
import numpy as np
import math
import random
import cv2
from numpy.linalg import inv


class ImageInfo:
    def __init__(self, name, img, position):
        self.name = name
        self.img = img
        self.position = position

def computeMapping(leftImage, rightImage):
    leftGrey = cv2.cvtColor(leftImage, cv2.COLOR_BGR2GRAY)
    rightGrey = cv2.cvtColor(rightImage, cv2.COLOR_BGR2GRAY)

    # surf
    surf = cv2.xfeatures2d.SURF_create()
    leftKeypoints, leftDescriptors = surf.detectAndCompute(leftGrey, None)
    rightKeypoints, rightDescriptors = surf.detectAndCompute(rightGrey, None)
    bf = cv2.BFMatcher(cv2.NORM_L1,crossCheck=False)
    
    # # sift
    # sift = cv2.xfeatures2d.SIFT_create()
    # leftKeypoints, leftDescriptors = sift.detectAndCompute(leftGrey, None)
    # rightKeypoints, rightDescriptors = sift.detectAndCompute(rightGrey, None)
    # bf = cv2.BFMatcher(cv2.NORM_L1,crossCheck=False)
    
    # # orb
    # orb = cv2.ORB_create(nfeatures=1500)
    # leftKeypoints, leftDescriptors = orb.detectAndCompute(leftGrey, None)
    # rightKeypoints, rightDescriptors = orb.detectAndCompute(rightGrey, None)
    # bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    matches = bf.match(leftDescriptors, rightDescriptors)
    matches = sorted(matches, key=lambda x: x.distance)
    # Minimize matches
    nMatches = int( 20 * len(matches) / 100)
    if nMatches < 4:
        return None
    matches = matches[:nMatches]
    motionModel = 1
    nRANSAC = 500
    RANSACThreshold = 0.1

    alignMatrix,inmatches = alignPair(leftKeypoints, rightKeypoints, matches, motionModel, nRANSAC, RANSACThreshold) 
    print(f'* Matches: {len(inmatches)}')
    # Map feature pts in both images
    img3 = cv2.drawMatches(leftGrey,leftKeypoints,rightGrey,rightKeypoints,inmatches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.namedWindow('Features', cv2.WINDOW_NORMAL)
    cv2.imshow('Features',img3)
    cv2.waitKey(0)

    # # Map inliers
    # img3 = cv2.drawMatches(leftGrey,leftKeypoints,rightGrey,rightKeypoints,inlier_indices,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    # cv2.namedWindow('Inliers', cv2.WINDOW_NORMAL)
    # cv2.imshow('Inliers',img3)
    # cv2.waitKey(0)

    return alignMatrix

def alignPair(f1, f2, matches, m, nRANSAC, RANSACthresh):
    Sample_size = 4
    max_inlier = 0
    count_inlier = []

    for i in range(nRANSAC):
        H = np.eye(3)
        randMatch = np.random.choice(matches, Sample_size)
        H = computeHomography(f1,f2,randMatch)

        num_inliers = getInliers(f1, f2, matches, H, RANSACthresh)
        if len(num_inliers) > max_inlier:
            count_inlier = num_inliers
            max_inlier = len(num_inliers)

        M,inmatches = leastSquaresFit(f1, f2, matches, m, count_inlier)
    # print(f'* M,inmatches: {M, len(inmatches)}')
    return M,inmatches

def computeHomography(f1, f2, matches, A_out=None):
    num_matches = len(matches)
    # Dimensions of the A matrix in the homogenous linear equation Ah = 0
    num_rows = 2 * num_matches
    num_cols = 9
    A_matrix_shape = (num_rows,num_cols)
    A = np.zeros(A_matrix_shape)

    for i in range(len(matches)):
        m = matches[i]
        (a_x, a_y) = f1[m.queryIdx].pt
        (b_x, b_y) = f2[m.trainIdx].pt

        #Fill in the matrix A in this loop.
        #Access elements using square brackets. e.g. A[0,0]
        A[2*i, 0] = a_x
        A[2*i, 1] = a_y
        A[2*i, 2] = 1
        A[2*i, 3] = 0
        A[2*i, 4] = 0
        A[2*i, 5] = 0
        A[2*i, 6] = -b_x * a_x
        A[2*i, 7] = -b_x * a_y
        A[2*i, 8] = -b_x

        A[2*i+1,0] = 0
        A[2*i+1,1] = 0
        A[2*i+1,2] = 0
        A[2*i+1,3] = a_x
        A[2*i+1,4] = a_y
        A[2*i+1,5] = 1
        A[2*i+1,6] = -b_y * a_x
        A[2*i+1,7] = -b_y * a_y
        A[2*i+1,8] = -b_y

    U, s, Vt = np.linalg.svd(A)

    if A_out is not None:
        A_out[:] = A

    #s is a 1-D array of singular values sorted in descending order
    #U, Vt are unitary matrices
    #Rows of Vt are the eigenvectors of A^TA.
    #Columns of U are the eigenvectors of AA^T.

    #Homography to be calculated
    H = np.eye(3)

    #Fill the homography H with the appropriate elements of the SVD
    minsv = Vt.shape[0] - 1
    H = (Vt[minsv] / Vt[minsv][8]).reshape(3,3)

    return H

def getInliers(f1, f2, matches, M, RANSACthresh):
    inlier_indices = []

    for i in range(len(matches)):
        #Determine if the ith matched feature f1[id1], when transformed
        #by M, is within RANSACthresh of its match in f2.
        #If so, append i to inliers
        # features
        m = matches[i]
        (a_x, a_y) = f1[m.queryIdx].pt
        (b_x, b_y) = f2[m.trainIdx].pt

        # Transform 
        p = np.zeros(3)
        p[0] = a_x
        p[1] = a_y
        p[2] = 1
        translated = np.dot(M , p)
        xt = translated[0] / translated[2]
        yt = translated[1] / translated[2]

        # Euclidean distance
        x_diff = (b_x - xt) * (b_x - xt)
        y_diff = (b_y - yt) * (b_y - yt)
        distance = np.sqrt(x_diff + y_diff)
        if distance <= RANSACthresh:
        	inlier_indices.append(i)
    # print(f'*: Inliers: {len(inlier_indices)}')
    return inlier_indices

def leastSquaresFit(f1, f2, matches, m, inlier_indices):
    # This function needs to handle two possible motion models,
    # pure translations (eTranslate)
    # and full homographies (eHomography).
    M = np.eye(3)
        #Compute a homography M using all inliers.
        #This should call computeHomography.
    inmatches = []
    for i in range(len(inlier_indices)):
        inmatches.append(matches[inlier_indices[i]])
    M = computeHomography(f1,f2,inmatches)
    return M,inmatches

# if __name__ == '__main__':
#     images = [cv2.imread('yosemite1.jpg'), cv2.imread('yosemite2.jpg')]
#     t = computeMapping(images[0], images[1])#.dot(t)
#     print(t)


def blendImages(ipv, blendWidth=70, is360=False, A_out=None):
    accWidth, accHeight, channels, width, translation = getAccSize(ipv)
    acc = pasteImages(ipv, translation, blendWidth, accWidth, accHeight, channels)
    compImage = normalizeBlend(acc)
    # print(f'*: Normalized Image: {compImage}')
    cv2.imwrite('compImage.jpg',compImage)

    # Determine the final image width
    outputWidth = accWidth
    x_init, y_init, x_final, y_final = getDriftParams(ipv, translation, width)
    # Compute the affine transform
    A = np.identity(3)
    # Warp and crop the composite
    croppedImage = cv2.warpPerspective(compImage, A, (outputWidth, accHeight), flags=cv2.INTER_LINEAR)
    cv2.imwrite('croppedImage.jpg',croppedImage)
    return croppedImage

def getAccSize(ipv):
    # Compute bounding box for the mosaic
    minX = sys.maxsize
    minY = sys.maxsize
    maxX = 0
    maxY = 0
    channels = -1
    width = -1  # Assumes all images are the same width
    M = np.identity(3)
    for i in ipv:
        M = i.position
        img = i.img
        _, w, c = img.shape
        if channels == -1:
            channels = c
            width = w

        # BEGIN TODO 9
        # add some code here to update minX, ..., maxY
        #TODO-BLOCK-BEGIN
        tmp_minX, tmp_minY, tmp_maxX, tmp_maxY = imageBoundingBox(img,M)
        minX = min(tmp_minX, minX)
        minY = min(tmp_minY, minY)
        maxX = max(tmp_maxX, maxX)
        maxY = max(tmp_maxY, maxY)
        #TODO-BLOCK-END
        # END TODO

    # Create an accumulator image
    accWidth = int(math.ceil(maxX) - math.floor(minX))
    accHeight = int(math.ceil(maxY) - math.floor(minY))
    print ('accWidth, accHeight:', (accWidth, accHeight))
    translation = np.array([[1, 0, -minX], [0, 1, -minY], [0, 0, 1]])

    return accWidth, accHeight, channels, width, translation


def pasteImages(ipv, translation, blendWidth, accWidth, accHeight, channels):
    acc = np.zeros((accHeight, accWidth, channels + 1))
    # Add in all the images
    M = np.identity(3)
    for count, i in enumerate(ipv):
        M = i.position
        img = i.img

        M_trans = translation.dot(M)
        accumulateBlend(img, acc, M_trans, blendWidth)

    return acc

def normalizeBlend(acc):
    """
       INPUT:
         acc: input image whose alpha channel (4th channel) contains
         normalizing weight values
       OUTPUT:
         img: image with r,g,b values of acc normalized
    """
    # BEGIN TODO 11
    # fill in this routine..
    #TODO-BLOCK-BEGIN
    h_acc = acc.shape[0]
    w_acc = acc.shape[1]
    img = np.zeros((h_acc, w_acc, 3))
    for i in range(0, w_acc, 1):
        for j in range(0, h_acc, 1):
            if acc[j,i,3]>0:
                img[j,i,0] = int (acc[j,i,0] / acc[j,i,3])
                img[j,i,1] = int (acc[j,i,1] / acc[j,i,3])
                img[j,i,2] = int (acc[j,i,2] / acc[j,i,3])
            else:
                img[j,i,0] = 0
                img[j,i,1] = 0
                img[j,i,2] = 0
    img = np.uint8(img)
    #TODO-BLOCK-END
    # END TODO
    return img

def getDriftParams(ipv, translation, width):
    # Add in all the images
    M = np.identity(3)
    for count, i in enumerate(ipv):
        if count != 0 and count != (len(ipv) - 1):
            continue

        M = i.position

        M_trans = translation.dot(M)

        p = np.array([0.5 * width, 0, 1])
        p = M_trans.dot(p)

        # First image
        if count == 0:
            x_init, y_init = p[:2] / p[2]
        # Last image
        if count == (len(ipv) - 1):
            x_final, y_final = p[:2] / p[2]

    return x_init, y_init, x_final, y_final

def computeDrift(x_init, y_init, x_final, y_final, width):
    A = np.identity(3)
    drift = (float)(y_final - y_init)
    # We implicitly multiply by -1 if the order of the images is swapped...
    length = (float)(x_final - x_init)
    A[0, 2] = -0.5 * width
    # Negative because positive y points downwards
    A[1, 0] = -drift / length

    return A

def imageBoundingBox(img, M):
    """
       This is a useful helper function that you might choose to implement
       that takes an image, and a transform, and computes the bounding box
       of the transformed image.

       INPUT:
         img: image to get the bounding box of
         M: the transformation to apply to the img
       OUTPUT:
         minX: int for the minimum X value of a corner
         minY: int for the minimum Y value of a corner
         minX: int for the maximum X value of a corner
         minY: int for the maximum Y value of a corner
    """
    #TODO 8
    #TODO-BLOCK-BEGIN
    h = img.shape[0]
    w = img.shape[1]
    
    p1 = np.array([0, 0, 1])
    p2 = np.array([0, h-1, 1])
    p3 = np.array([w-1, 0, 1])
    p4 = np.array([w-1, h-1, 1])
    
    p1 = np.dot(M,p1)
    p2 = np.dot(M,p2)
    p3 = np.dot(M,p3)
    p4 = np.dot(M,p4)

    def roundup(x):
        if x < 0.0:
            x = math.floor(x)
        else:
            x = math.ceil(x)
        return x
    
    # normalize when converting homogeneous coordinates back to Cartesian coordinates
    # When Z is not 0 the point represented is the point (X/Z, Y/Z) in the Euclidean plane
    
    minX = roundup(min(p1[0]/p1[2], p2[0]/p2[2], p3[0]/p3[2], p4[0]/p4[2]))
    minY = roundup(min(p1[1]/p1[2], p2[1]/p2[2], p3[1]/p3[2], p4[1]/p4[2]))
    maxX = roundup(max(p1[0]/p1[2], p2[0]/p2[2], p3[0]/p3[2], p4[0]/p4[2]))
    maxY = roundup(max(p1[1]/p1[2], p2[1]/p2[2], p3[1]/p3[2], p4[1]/p4[2]))
    
    return int(minX), int(minY), int(maxX), int(maxY)

def accumulateBlend(img, acc, M, blendWidth):
    """
       INPUT:
         img: image to add to the accumulator
         acc: portion of the accumulated image where img should be added
         M: the transformation mapping the input image to the accumulator
         blendWidth: width of blending function. horizontal hat function
       OUTPUT:
         modify acc with weighted copy of img added where the first
         three channels of acc record the weighted sum of the pixel colors
         and the fourth channel of acc records a sum of the weights
    """
    # BEGIN TODO 10
    # Fill in this routine
    #TODO-BLOCK-BEGIN
    h = img.shape[0]
    w = img.shape[1]
    
    h_acc = acc.shape[0]
    w_acc = acc.shape[1]
    
    ## get the bounding box of img in acc
    minX, minY, maxX, maxY = imageBoundingBox(img,M)
  
    for i in range(minX,maxX,1):
        for j in range(minY,maxY,1):
            # don't want to include black pixels when inverse warping
            ## whether current pixel black or white
            p = np.array([i, j, 1.])
            p = np.dot(inv(M),p)
            newx = min(p[0] / p[2], w-1)
            newy = min(p[1] / p[2], h-1)
            
            if newx <0 or newx >= w or newy < 0 or newy >= h:
                continue
            if img[int(newy), int(newx), 0] == 0 and img[int(newy), int(newx), 1] ==0 and img[int(newy), int(newx), 2] == 0:
                continue
            if newx >= 0 and newx < w-1 and newy >= 0 and newy < h-1:    
                weight = 1.0
                if newx >= minX and newx < minX + blendWidth:
                    weight = 1. * (newx - minX) / blendWidth
                if newx <= maxX and newx > maxX - blendWidth:
                    weight = 1. * (maxX - newx) / blendWidth
                acc[j,i,3] += weight
            
                for k in range(3):
                    acc[j,i,k] += img[int(newy),int(newx),k] * weight      

if __name__ == '__main__':
    # images = [cv2.imread('yosemite1.jpg'), cv2.imread('yosemite2.jpg'), cv2.imread('yosemite3.jpg'), cv2.imread('yosemite4.jpg')]
    images = [cv2.imread('carmel1.png'), cv2.imread('carmel2.png'), cv2.imread('carmel3.png'), cv2.imread('carmel4.png')]

    # Generate 
    t = np.eye(3); ipv = []; inmatches = [];
    for i in range(0,len(images)-1):
        ipv.append(ImageInfo('',images[i], np.linalg.inv(t)))
        t = computeMapping(images[i], images[i+1]).dot(t)
    ipv.append(ImageInfo('',images[len(images)-1], np.linalg.inv(t)))
    print(f'Mapping Matrix:\n {t}')
    # Blend Image
    blendImages(ipv)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv_contrib\modules\xfeatures2d\src\surf.cpp:1029: error: (-213:The function/feature is not implemented) This algorithm is patented and is excluded in this configuration; Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in function 'cv::xfeatures2d::SURF::create'
